In [1]:
import pickle
import numpy as np
import os
import torch
import json
import shlex
import pickle
import time
import subprocess
import numpy as np
import torch.utils.data as data
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import datasets.data_utils as d_utils
import pandas as pd

from sklearn.metrics import roc_auc_score,confusion_matrix
from sklearn.metrics import accuracy_score
from utils.util import AverageMeter, shapenetpart_metrics
from models.backbones import ResNet
from models.heads import ClassifierResNet, MultiPartSegHeadResNet, SceneSegHeadResNet
from models.losses import LabelSmoothingCrossEntropyLoss, MultiShapeCrossEntropy, MaskedCrossEntropy
from utils.config import config, update_config
from utils.lr_scheduler import get_scheduler
from torchvision import transforms
from sklearn.metrics import roc_curve, auc
from IPython.display import clear_output
import pickle

def config_seting(cfg = 'cfgs/brain/brain.yaml'):
    update_config(cfg)
    return config


# Usefull functional for dataprocesing, train and validation

In [2]:

class BrainDataSeg():
    def __init__(self, data_type = 'train', num_points = 2048, 
                 transforms=None,
                 data_post = '',
                 datafolder = 'BrainData'):
        self.num_points = num_points
        self.transforms = transforms
        if data_type == 'test':
            filename = f'data/{datafolder}/test_data{data_post}.pkl'
        if data_type == 'train':
            filename = f'data/{datafolder}/trainval_data{data_post}.pkl'
        with open(filename, 'rb') as f:
            self.points, self.points_labels, self.labels = pickle.load(f)
        print(f"{filename} loaded successfully")

    def __getitem__(self, idx):
        current_points = self.points[idx]
        current_points_labels = self.points_labels[idx]
        cur_num_points = current_points.shape[0]
        if cur_num_points >= self.num_points:
            choice = np.random.choice(cur_num_points, self.num_points)
            current_points = current_points[choice, :]
            current_points_labels = current_points_labels[choice]
            mask = torch.ones(self.num_points).type(torch.int32)
        else:
            padding_num = self.num_points - cur_num_points
            shuffle_choice = np.random.permutation(np.arange(cur_num_points))
            padding_choice = np.random.choice(cur_num_points, padding_num)
            choice = np.hstack([shuffle_choice, padding_choice])
            current_points = current_points[choice, :]
            current_points_labels = current_points_labels[choice]
            mask = torch.cat([torch.ones(cur_num_points), torch.zeros(padding_num)]).type(torch.int32)
        if self.transforms is not None:
            current_points = self.transforms(current_points)
        label = torch.from_numpy(self.labels[idx]).type(torch.int64)
        current_points_labels = torch.from_numpy(current_points_labels).type(torch.int64)

        return current_points, mask, current_points_labels, label

    def __len__(self):
        return len(self.points)
def get_loader(num_points,batch_size = 16,data_post = '', datafolder = 'BrainData'):
    trans = transforms.Compose([d_utils.PointcloudToTensor()])
    train_dataset = BrainDataSeg(num_points=num_points,
                                 data_type = 'train',
                                 transforms = trans, 
                                 data_post = data_post,
                                 datafolder = datafolder
                                )
    test_dataset = BrainDataSeg(num_points=num_points,
                                data_type = 'test',
                                transforms = trans, 
                                data_post = data_post,
                               datafolder = datafolder)
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               shuffle=False,
                                               pin_memory=True,
                                               drop_last=True)

    test_loader = torch.utils.data.DataLoader(test_dataset,
                                              batch_size=batch_size,
                                              shuffle=False,
                                              pin_memory=True,
                                              drop_last=False)

    return train_loader, test_loader
class MultiPartSegmentationModel(nn.Module):
    def __init__(self, config, backbone, head, num_classes, num_parts,
                 input_features_dim, radius, sampleDl, nsamples, npoints,
                 width=144, depth=2, bottleneck_ratio=2):
        super(MultiPartSegmentationModel, self).__init__()
        if backbone == 'resnet':
            self.backbone = ResNet(config, input_features_dim, radius, sampleDl, nsamples, npoints,
                                   width=width, depth=depth, bottleneck_ratio=bottleneck_ratio)
        else:
            raise NotImplementedError(f"Backbone {backbone} not implemented in Multi-Part Segmentation Model")

        if head == 'resnet_part_seg':
            self.segmentation_head = MultiPartSegHeadResNet(num_classes, width, radius, nsamples, num_parts)
        else:
            raise NotImplementedError(f"Head {backbone} not implemented in Multi-Part Segmentation Model")

    def forward(self, xyz, mask, features):
        end_points = self.backbone(xyz, mask, features)
        return self.segmentation_head(end_points)

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Conv1d):
                torch.nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    torch.nn.init.zeros_(m.bias)
def build_multi_part_segmentation(config, weights = None):
    model = MultiPartSegmentationModel(config, config.backbone, config.head, config.num_classes, config.num_parts,
                                       config.input_features_dim,
                                       config.radius, config.sampleDl, config.nsamples, config.npoints,
                                       config.width, config.depth, config.bottleneck_ratio)
    criterion = MultiShapeCrossEntropy(config.num_classes, weights)
    return model, criterion
def Find_Optimal_Cutoff(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Parameters
    ----------
    target : Matrix with dependent or target data, where rows are observations

    predicted : Matrix with predicted data, where rows are observations

    Returns
    -------     
    list type, with optimal cutoff value
        
    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.iloc[(roc.tf-0).abs().argsort()[:1]]

    return list(roc_t['threshold'])

def train(epoch, train_loader, model, criterion, optimizer, scheduler, config):
    """
    One epoch training
    """
    model.train()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    loss_meter = AverageMeter()
    end = time.time()
    pred_soft_flats = []
    points_labels_flats = []
    for idx, (points, mask, points_labels, shape_labels) in enumerate(train_loader):
        data_time.update(time.time() - end)
        bsz = points.size(0)
        # forward
        features = points
        features = features.transpose(1, 2).contiguous()

        points = points.cuda(non_blocking=True)
        mask = mask.cuda(non_blocking=True)
        features = features.cuda(non_blocking=True)
        points_labels = points_labels.cuda(non_blocking=True)
        shape_labels = shape_labels.cuda(non_blocking=True)
        pred = model(points, mask, features)
        loss = criterion(pred, points_labels, shape_labels)
        
        m = torch.nn.Softmax(dim=1)
        pred_soft_flats += list(np.array(m(pred[0])[:,1,:].reshape(-1).detach().cpu()))
        points_labels_flats += list(np.array(points_labels.reshape(-1).detach().cpu()))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        # update meters
        loss_meter.update(loss.item(), bsz)
        batch_time.update(time.time() - end)
        end = time.time()

    print('roc train',roc_auc_score(points_labels_flats,pred_soft_flats))
    opt = Find_Optimal_Cutoff(points_labels_flats, pred_soft_flats)
    print('optimal cut',opt)

    return loss_meter.avg, opt

def validate(epoch, test_loader, model, criterion, config, num_votes=10, is_conf = False):
    """
    One epoch validating
    """
    batch_time = AverageMeter()
    losses = AverageMeter()

    model.eval()
    with torch.no_grad():
        all_logits = []
        all_points_labels = []
        all_shape_labels = []
        all_masks = []
        end = time.time()
        TS = d_utils.BatchPointcloudScaleAndJitter(scale_low=config.scale_low,
                                                   scale_high=config.scale_high,
                                                   std=config.noise_std,
                                                   clip=config.noise_clip)
        pred_soft_flats = []
        points_labels_flats = []
        for idx, (points_orig, mask, points_labels, shape_labels) in enumerate(test_loader):
            vote_logits = None
            vote_points_labels = None
            vote_shape_labels = None
            vote_masks = None
            for v in range(num_votes):
                batch_logits = []
                batch_points_labels = []
                batch_shape_labels = []
                batch_masks = []
                # augment for voting
                if v > 0:
                    points = TS(points_orig)
                else:
                    points = points_orig
                # forward
                features = points
                features = features.transpose(1, 2).contiguous()
                points = points.cuda(non_blocking=True)
                mask = mask.cuda(non_blocking=True)
                features = features.cuda(non_blocking=True)
                points_labels = points_labels.cuda(non_blocking=True)
                shape_labels = shape_labels.cuda(non_blocking=True)

                pred = model(points, mask, features)
                loss = criterion(pred, points_labels, shape_labels)
                losses.update(loss.item(), points.size(0))
                
                m = torch.nn.Softmax(dim=1)
                pred_soft_flats += list(np.array(m(pred[0])[:,1,:].reshape(-1).detach().cpu()))
                points_labels_flats += list(np.array(points_labels.reshape(-1).detach().cpu()))
                

                # collect
                bsz = points.shape[0]
                for ib in range(bsz):
                    sl = shape_labels[ib]
                    logits = pred[sl][ib]
                    pl = points_labels[ib]
                    pmk = mask[ib]
                    batch_logits.append(logits.cpu().numpy())
                    batch_points_labels.append(pl.cpu().numpy())
                    batch_shape_labels.append(sl.cpu().numpy())
                    batch_masks.append(pmk.cpu().numpy().astype(np.bool))

                # measure elapsed time
                batch_time.update(time.time() - end)
                end = time.time()

                if vote_logits is None:
                    vote_logits = batch_logits
                    vote_points_labels = batch_points_labels
                    vote_shape_labels = batch_shape_labels
                    vote_masks = batch_masks
                else:
                    for i in range(len(vote_logits)):
                        vote_logits[i] = vote_logits[i] + (batch_logits[i] - vote_logits[i]) / (v + 1)

            all_logits += vote_logits
            all_points_labels += vote_points_labels
            all_shape_labels += vote_shape_labels
            all_masks += vote_masks
            if idx % config.print_freq == 0:
                print(
                    f'V{num_votes} Test: [{idx}/{len(test_loader)}]\t'
                    f'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                    f'Loss {losses.val:.4f} ({losses.avg:.4f})')
                

        print('roc test',roc_auc_score(points_labels_flats,pred_soft_flats))
            
        if is_conf:
            acc, shape_ious, msIoU, mIoU, confs = shapenetpart_metrics(config.num_classes,
                                                            config.num_parts,
                                                            all_shape_labels,
                                                            all_logits,
                                                            all_points_labels,
                                                            all_masks, is_conf = is_conf)
        else:
            acc, shape_ious, msIoU, mIoU = shapenetpart_metrics(config.num_classes,
                                                            config.num_parts,
                                                            all_shape_labels,
                                                            all_logits,
                                                            all_points_labels,
                                                            all_masks, is_conf = is_conf)
            
    if is_conf:
        return losses.avg,acc, msIoU, mIoU, confs
    else:
        return losses.avg,acc, msIoU, mIoU

# Hyperparameters

In [3]:
# Brain Data FCD
WEIGHTS = [1,167]
CFG = 'cfgs/brain/brain3exp.yaml'
IS_CONF = True
DATAFOLDER = 'BrainData'

# Data processing, train and validation

In [4]:
config = config_seting(CFG)

/home/guest/CloserLook3D/pytorch/utils/config.py:108: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  exp_config = edict(yaml.load(f))


In [5]:
model_, criterion = build_multi_part_segmentation(config, WEIGHTS)
criterion.cuda()

MultiShapeCrossEntropy()

In [6]:
res_dict = {}
for e in range(15):
    DATA_POSTFIX = f'_3exp_{e}'
    train_loader, test_loader = get_loader(num_points = config.num_points, 
                                       data_post = DATA_POSTFIX, 
                                       datafolder = DATAFOLDER )
    
    
    n_data = len(train_loader.dataset)
    print(f"length of training dataset: {n_data}")
    n_data = len(test_loader.dataset)
    print(f"length of testing dataset: {n_data}")
    
    model = model_
    model.cuda()
    if config.optimizer == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(),
                                        lr=config.batch_size * dist.get_world_size() / 16 * config.base_learning_rate,
                                        momentum=config.momentum,
                                        weight_decay=config.weight_decay)
    elif config.optimizer == 'adam':
        optimizer = torch.optim.Adam(model.parameters(),
                                     lr=config.base_learning_rate,
                                     weight_decay=config.weight_decay)
    elif config.optimizer == 'adamW':
        optimizer = torch.optim.AdamW(model.parameters(),
                                      lr=config.base_learning_rate,
                                      weight_decay=config.weight_decay)
    else:
        raise NotImplementedError(f"Optimizer {config.optimizer} not supported")
    scheduler = get_scheduler(optimizer, len(train_loader), config)
    
    res_dict[e] = {}
    train_losses = []
    test_iou = []
    test_losses = []

    for epoch in range(config.start_epoch, config.epochs + 1):

        tic = time.time()
        loss, opt = train(epoch, train_loader, model, criterion, optimizer, scheduler, config)

        print('epoch {}, total time {:.2f}, lr {:.5f}'.format(epoch,
                                                                    (time.time() - tic),
                                                                    optimizer.param_groups[0]['lr']))
        tmp = validate(epoch, test_loader, model, criterion, config, num_votes=1, is_conf = IS_CONF)
        if IS_CONF:
            loss_test,acc, msIoU, mIoU,confs = tmp
        else:
            loss_test,acc, msIoU, mIoU = tmp


        print('test_loss', loss_test)
        print('ins_loss', loss)
        print('test_acc', acc)
        print('msIoU', msIoU)
        print('mIoU', mIoU)
        if IS_CONF:
            print(confs)
        test_losses.append(loss_test)
        train_losses.append(loss)
        test_iou.append(msIoU)

        res_dict[e] = {'test_loss':test_losses,'train_loss':train_losses,'iou':test_iou}
        with open('3_exp_output.pkl', 'wb') as handle:
            pickle.dump(res_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open('3_exp_output.pkl', 'rb') as handle:
            res_dict = pickle.load(handle)
        
        if epoch > 1:
            if res_dict[e]['test_loss'][-1] < np.min(res_dict[e]['test_loss'][:-1]):
                torch.save(model.state_dict(), f'3_exp_{e}_model.pkl')

data/BrainData/trainval_data_3exp_0.pkl loaded successfully
data/BrainData/test_data_3exp_0.pkl loaded successfully
length of training dataset: 140
length of testing dataset: 10
roc train 0.8879578967805929
optimal cut [0.6213774681091309]
epoch 1, total time 41.24, lr 0.00199
V1 Test: [0/1]	Time 3.742 (3.742)	Loss 0.6911 (0.6911)
roc test 0.8586773284158296
test_loss 0.6911492347717285
ins_loss 0.5705680251121521
test_acc 0.994208984375
msIoU 0.0
mIoU 0.0
[[203614      0]
 [  1186      0]]
roc train 0.941085143334093
optimal cut [0.548254668712616]
epoch 2, total time 46.62, lr 0.00197
V1 Test: [0/1]	Time 3.699 (3.699)	Loss 0.7023 (0.7023)
roc test 0.7831373603357658
test_loss 0.7023193836212158
ins_loss 0.3642831817269325
test_acc 0.993984375
msIoU 0.0
mIoU 0.0
[[203568      0]
 [  1232      0]]
roc train 0.9470227383492312
optimal cut [0.540587306022644]
epoch 3, total time 46.05, lr 0.00194
V1 Test: [0/1]	Time 3.432 (3.432)	Loss 0.6266 (0.6266)
roc test 0.8255185610423061
test_loss

roc train 0.9858693605003545
optimal cut [0.5709178447723389]
epoch 24, total time 46.07, lr 0.00003
V1 Test: [0/1]	Time 2.142 (2.142)	Loss 0.3168 (0.3168)
roc test 0.9484359810664185
test_loss 0.3168443739414215
ins_loss 0.14182406105101109
test_acc 0.913984375
msIoU 0.059844867505702906
mIoU 0.059844867505702906
[[186133  17408]
 [   208   1051]]
roc train 0.9852233871035971
optimal cut [0.5605823993682861]
epoch 25, total time 45.13, lr 0.00001
V1 Test: [0/1]	Time 3.647 (3.647)	Loss 0.3522 (0.3522)
roc test 0.9365506804010343
test_loss 0.3522241413593292
ins_loss 0.14002611860632896
test_acc 0.9089697265625
msIoU 0.05214338387453714
mIoU 0.05214338387453714
[[185158  18373]
 [   270    999]]
data/BrainData/trainval_data_3exp_1.pkl loaded successfully
data/BrainData/test_data_3exp_1.pkl loaded successfully
length of training dataset: 140
length of testing dataset: 10
roc train 0.9647467766006392
optimal cut [0.49469423294067383]
epoch 1, total time 46.24, lr 0.00199
V1 Test: [0/1]	Ti

V1 Test: [0/1]	Time 3.730 (3.730)	Loss 1.4380 (1.4380)
roc test 0.8977629450927854
test_loss 1.4379956722259521
ins_loss 0.10803941264748573
test_acc 0.9812060546875
msIoU 0.0001584786051371179
mIoU 0.0001584786051371179
[[200950   2723]
 [  1126      1]]
roc train 0.9903003314318553
optimal cut [0.5374000072479248]
epoch 23, total time 40.54, lr 0.00007
V1 Test: [0/1]	Time 3.740 (3.740)	Loss 1.3968 (1.3968)
roc test 0.9089327061971497
test_loss 1.3967846632003784
ins_loss 0.10741105116903782
test_acc 0.9789990234375
msIoU 0.0070247933787537286
mIoU 0.0070247933787537286
[[200448   3208]
 [  1093     51]]
roc train 0.988845864088536
optimal cut [0.4569869935512543]
epoch 24, total time 46.23, lr 0.00003
V1 Test: [0/1]	Time 3.679 (3.679)	Loss 1.3731 (1.3731)
roc test 0.9084956707862966
test_loss 1.3731409311294556
ins_loss 0.11594884376972914
test_acc 0.9775341796875
msIoU 0.010373443961880821
mIoU 0.010373443961880821
[[200149   3408]
 [  1193     50]]
roc train 0.9912412247930715
opti

roc train 0.9952009632274254
optimal cut [0.5252717733383179]
epoch 20, total time 46.26, lr 0.00025
V1 Test: [0/1]	Time 3.642 (3.642)	Loss 1.3241 (1.3241)
roc test 0.8830215159374284
test_loss 1.3240784406661987
ins_loss 0.07771602924913168
test_acc 0.9486328125
msIoU 0.030214624150583458
mIoU 0.030214624150583458
[[193896   9409]
 [  1111    384]]
roc train 0.9944018259123985
optimal cut [0.3895890414714813]
epoch 21, total time 46.28, lr 0.00018
V1 Test: [0/1]	Time 3.625 (3.625)	Loss 1.3238 (1.3238)
roc test 0.8842408044045857
test_loss 1.3237583637237549
ins_loss 0.08683465793728828
test_acc 0.94908203125
msIoU 0.021060542390244824
mIoU 0.021060542390244824
[[194063   9256]
 [  1172    309]]
roc train 0.9956101258022891
optimal cut [0.548551082611084]
epoch 22, total time 40.60, lr 0.00012
V1 Test: [0/1]	Time 3.784 (3.784)	Loss 1.5508 (1.5508)
roc test 0.8852788191595494
test_loss 1.550776481628418
ins_loss 0.07605934189632535
test_acc 0.9592431640625
msIoU 0.015300643696636404
mIo

roc train 0.9887488552182309
optimal cut [0.41005638241767883]
epoch 18, total time 40.37, lr 0.00043
V1 Test: [0/1]	Time 3.762 (3.762)	Loss 0.3012 (0.3012)
roc test 0.8647382103410278
test_loss 0.3012363612651825
ins_loss 0.11661962699145079
test_acc 0.977998046875
msIoU 0.0
mIoU 0.0
[[200294   4428]
 [    78      0]]
roc train 0.9937764526677834
optimal cut [0.6808142066001892]
epoch 19, total time 46.55, lr 0.00034
V1 Test: [0/1]	Time 3.731 (3.731)	Loss 0.2711 (0.2711)
roc test 0.8922430954038775
test_loss 0.27105340361595154
ins_loss 0.08578514214605093
test_acc 0.978359375
msIoU 0.0009054992543477925
mIoU 0.0009054992543477925
[[200361   4360]
 [    72      7]]
roc train 0.9930353624679231
optimal cut [0.583744466304779]
epoch 20, total time 42.19, lr 0.00025
V1 Test: [0/1]	Time 3.639 (3.639)	Loss 0.2693 (0.2693)
roc test 0.889288020755219
test_loss 0.26928356289863586
ins_loss 0.09094206243753433
test_acc 0.9736083984375
msIoU 0.0
mIoU 0.0
[[199395   5324]
 [    81      0]]
roc t

...

roc train 0.9975795039729665
optimal cut [0.670346200466156]
epoch 6, total time 46.25, lr 0.00175
V1 Test: [0/1]	Time 3.694 (3.694)	Loss 0.6238 (0.6238)
roc test 0.9829802901153629
test_loss 0.6237620711326599
ins_loss 0.04213973227888346
test_acc 0.9970361328125
msIoU 0.0
mIoU 0.0
[[204193    316]
 [   291      0]]
roc train 0.9977625933415213
optimal cut [0.6311137080192566]
epoch 7, total time 46.29, lr 0.00166
V1 Test: [0/1]	Time 1.924 (1.924)	Loss 0.5608 (0.5608)
roc test 0.9884011268437148
test_loss 0.5608413219451904
ins_loss 0.03980408934876323
test_acc 0.9974951171875
msIoU 0.02621432880537013
mIoU 0.02621432880537013
[[204261    216]
 [   297     26]]
roc train 0.9981178893118375
optimal cut [0.6604161858558655]
epoch 8, total time 45.62, lr 0.00157
V1 Test: [0/1]	Time 3.757 (3.757)	Loss 0.7686 (0.7686)
roc test 0.9749347622915125
test_loss 0.7685763239860535
ins_loss 0.03571010613813996
test_acc 0.9973779296875
msIoU 0.009708737769818081
mIoU 0.009708737769818081
[[204253  

roc train 0.9969771924864619
optimal cut [0.6085959672927856]
epoch 4, total time 45.36, lr 0.00189
V1 Test: [0/1]	Time 3.809 (3.809)	Loss 0.4513 (0.4513)
roc test 0.9716039913380856
test_loss 0.4512573778629303
ins_loss 0.04676699684932828
test_acc 0.9541748046875
msIoU 0.14968819020024046
mIoU 0.14968819020024046
[[193760   8980]
 [   405   1655]]
roc train 0.9969804003546402
optimal cut [0.5679717659950256]
epoch 5, total time 46.22, lr 0.00182
V1 Test: [0/1]	Time 3.772 (3.772)	Loss 0.1784 (0.1784)
roc test 0.982874861829379
test_loss 0.17840050160884857
ins_loss 0.04718549083918333
test_acc 0.9485986328125
msIoU 0.16400835810604536
mIoU 0.16400835810604536
[[192239  10409]
 [   118   2034]]
roc train 0.997532021651977
optimal cut [0.5709659457206726]
epoch 6, total time 41.04, lr 0.00175
V1 Test: [0/1]	Time 3.841 (3.841)	Loss 0.1389 (0.1389)
roc test 0.9897954098021453
test_loss 0.13892441987991333
ins_loss 0.04142735432833433
test_acc 0.958359375
msIoU 0.19159459443078883
mIoU 0.1

roc train 0.9971278307492782
optimal cut [0.48429548740386963]
epoch 2, total time 39.74, lr 0.00197
V1 Test: [0/1]	Time 3.706 (3.706)	Loss 0.1426 (0.1426)
roc test 0.9928427413576615
test_loss 0.14263199269771576
ins_loss 0.05261113168671727
test_acc 0.9835986328125
msIoU 0.19046916342167483
mIoU 0.19046916342167483
[[200765   3256]
 [   103    676]]
roc train 0.9975023852639251
optimal cut [0.6156817078590393]
epoch 3, total time 46.29, lr 0.00194
V1 Test: [0/1]	Time 3.779 (3.779)	Loss 0.3980 (0.3980)
roc test 0.983893363855816
test_loss 0.3979983329772949
ins_loss 0.040492646396160126
test_acc 0.980712890625
msIoU 0.1296008494228692
mIoU 0.1296008494228692
[[200298   3718]
 [   232    552]]
roc train 0.9975348983054607
optimal cut [0.4583510458469391]
epoch 4, total time 44.06, lr 0.00189
V1 Test: [0/1]	Time 2.742 (2.742)	Loss 0.1306 (0.1306)
roc test 0.9896562817628245
test_loss 0.13057750463485718
ins_loss 0.04543523769825697
test_acc 0.96857421875
msIoU 0.10722335344170024
mIoU 0

test_loss 0.1821049302816391
ins_loss 0.025550571735948324
test_acc 0.9837353515625
msIoU 0.20018287540340926
mIoU 0.20018287540340926
[[200780   3199]
 [   132    689]]
data/BrainData/trainval_data_3exp_14.pkl loaded successfully
data/BrainData/test_data_3exp_14.pkl loaded successfully
length of training dataset: 140
length of testing dataset: 10
roc train 0.9974542461059527
optimal cut [0.5749034285545349]
epoch 1, total time 43.67, lr 0.00199
V1 Test: [0/1]	Time 3.717 (3.717)	Loss 0.4485 (0.4485)
roc test 0.9876448191444552
test_loss 0.448484867811203
ins_loss 0.0406719078309834
test_acc 0.995390625
msIoU 0.08262260040820946
mIoU 0.08262260040820946
[[203793    739]
 [   205     63]]
roc train 0.9978752607986809
optimal cut [0.6762204766273499]
epoch 2, total time 46.76, lr 0.00197
V1 Test: [0/1]	Time 3.744 (3.744)	Loss 0.0752 (0.0752)
roc test 0.9946167858300605
test_loss 0.07523852586746216
ins_loss 0.040239554829895496
test_acc 0.9870703125
msIoU 0.10507278827535069
mIoU 0.105072

test_loss 0.13584093749523163
ins_loss 0.02692660060711205
test_acc 0.9933251953125
msIoU 0.0898035557744275
mIoU 0.0898035557744275
[[203283   1270]
 [    97    150]]
roc train 0.9988626205295249
optimal cut [0.7224020957946777]
epoch 24, total time 41.33, lr 0.00003
V1 Test: [0/1]	Time 3.728 (3.728)	Loss 0.0792 (0.0792)
roc test 0.9960285581570786
test_loss 0.07916678488254547
ins_loss 0.02616290422156453
test_acc 0.9913623046875
msIoU 0.13319856477072173
mIoU 0.13319856477072173
[[202795   1725]
 [    44    236]]
roc train 0.9988384826476376
optimal cut [0.7229582667350769]
epoch 25, total time 46.51, lr 0.00001
V1 Test: [0/1]	Time 3.726 (3.726)	Loss 0.0760 (0.0760)
roc test 0.9962463832090745
test_loss 0.0760270208120346
ins_loss 0.02535930066369474
test_acc 0.9919189453125
msIoU 0.13162557963379062
mIoU 0.13162557963379062
[[202899   1618]
 [    37    246]]
